In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from Secrets import CLIENT_ID, CLIENT_SECRET, REDIRECT_URL
import pandas as pd
import chromadb
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                               client_secret=CLIENT_SECRET,
                                               redirect_uri=REDIRECT_URL,
                                               scope="user-library-read"))

chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(name="spotify_data")



In [8]:
#6meRpNHvKC1VcBl5MIbVxo - hehe
#1Ob3QykC4dzf3rQwWxE9LV - study


playlist_id = '6meRpNHvKC1VcBl5MIbVxo'
playlist_length = sp.playlist_items(playlist_id)['total']

track_ids = []
for i in range(playlist_length // 100 + 1):
    playlist = sp.playlist_tracks(playlist_id, market = 'CAN',fields='items', limit=100, offset = i * 100 )
    for track in playlist['items']:
        if track['track'] is not None and track['track']['id'] is not None:
            track_ids.append(track['track']['id'])

playlist_audio_data = []

for id in track_ids:
    all_audio_data = list(sp.audio_features(id)[0].values())
    print(all_audio_data)
    audio_features = all_audio_data[:11] + all_audio_data[-1:]
    playlist_audio_data.append(audio_features)



[0.529, 0.673, 0, -4.711, 1, 0.175, 0.000307, 2.41e-06, 0.093, 0.366, 165.921, 'audio_features', '3F5CgOj3wFlRv51JsHbxhe', 'spotify:track:3F5CgOj3wFlRv51JsHbxhe', 'https://api.spotify.com/v1/tracks/3F5CgOj3wFlRv51JsHbxhe', 'https://api.spotify.com/v1/audio-analysis/3F5CgOj3wFlRv51JsHbxhe', 218365, 4]
[0.849, 0.424, 5, -9.579, 0, 0.324, 0.0635, 0, 0.0834, 0.153, 145.887, 'audio_features', '2BcMwX1MPV6ZHP4tUT9uq6', 'spotify:track:2BcMwX1MPV6ZHP4tUT9uq6', 'https://api.spotify.com/v1/tracks/2BcMwX1MPV6ZHP4tUT9uq6', 'https://api.spotify.com/v1/audio-analysis/2BcMwX1MPV6ZHP4tUT9uq6', 242966, 4]
[0.585, 0.909, 8, -6.474, 1, 0.0707, 0.0891, 9.7e-05, 0.119, 0.758, 93.372, 'audio_features', '3CA9pLiwRIGtUBiMjbZmRw', 'spotify:track:3CA9pLiwRIGtUBiMjbZmRw', 'https://api.spotify.com/v1/tracks/3CA9pLiwRIGtUBiMjbZmRw', 'https://api.spotify.com/v1/audio-analysis/3CA9pLiwRIGtUBiMjbZmRw', 210747, 4]
[0.84, 0.744, 1, -4.751, 1, 0.113, 0.0181, 0, 0.629, 0.171, 83.737, 'audio_features', '2tUL6dZf1mywCj5WvC

## Finding the optimal amount of clusters for KMeans with the elbow method

In [4]:
# intertias = []
# for i in range(1,10):
#     kmeans = KMeans(n_clusters=i)
#     kmeans.fit(playlist_audio_data)
#     intertias.append(kmeans.inertia_)

# plt.plot(range(1,10), intertias, marker = 'o')
# plt.title = ('Elbow method')
# plt.xlabel('Number of clusters')
# plt.ylabel('Inertia')
# plt.show()

In [5]:
kmeans = KMeans(n_clusters=6)
kmeans.fit(playlist_audio_data)

KMeans(n_clusters=6)

In [7]:
df = pd.read_csv('./dataset.csv')
# print(df.head())
df = df.drop(['artists', 'album_name', 'track_name', 'popularity', 'explicit', 'duration_ms', 'track_genre'], axis='columns')
df = df[df.columns[1:]]

# Convert DataFrame to a list of dictionaries
df_audio_data = df.values.tolist()
# for i in range(len(df_audio_data)):
#     collection.upsert(embeddings=df_audio_data[i][1:], ids=df_audio_data[i][0])

for centre in kmeans.cluster_centers_:
    results = collection.query(query_embeddings=centre.tolist(), n_results=4)
    for id in results['ids'][0]:
        print(sp.track(id)['name'])


Never Live Without You
U MI LUV
Go Your Own Way - 2018 Remaster
Go Your Own Way - 2018 Remaster
Durdur Beni
For Whom The Bell Tolls
Arrullo De Estrellas
Arrullo De Estrellas
Liga Pro 190
Seclusion
Six Days - Remix
I Love It Loud
how deep?
Es Tarde
All I Need
Hoyna Hoyna
All These Things
Melancholia [Mix Cut] - Original Mix
This Time
OnlyFans
Never in My Wildest Dreams
Em Cada Amanhecer
Hell Or High Water
愛得太遲
